In [1]:
import sqlite3
import pandas as pd
import numpy as np
import json
import re

from pathlib import Path

from pathlib import Path
import sys
BASE_DIR = Path().resolve().parent.parent
sys.path.append((BASE_DIR / 'vietnamadminunits/parser').as_posix())

from utils import check_point_in_polygon, find_nearest_point

BASE_DIR = Path().resolve().parent.parent

import warnings
warnings.filterwarnings('ignore')

# Adding Google Location for 63 provinces

In [2]:
df_province = pd.read_csv(BASE_DIR / 'data/raw/63_provinces_location_province_2025-07-24.csv')
df_district = pd.read_csv(BASE_DIR / 'data/raw/63_provinces_location_district_2025-07-25.csv')
df_ward = pd.read_csv(BASE_DIR / 'data/raw/63_provinces_location_ward_2025-07-24.csv')

In [3]:
col_name = {
    'address': 'province',
    'latitude': 'provinceLat',
    'longitude': 'provinceLon',
    'bounds': 'provinceBounds',
    'geo_address': 'provinceGeoAddress',
}
df_province.rename(columns=col_name, inplace=True)

In [4]:
col_name = {
    'address': 'districtAddress',
    'latitude': 'districtLat',
    'longitude': 'districtLon',
    'bounds': 'districtBounds',
    'geo_address': 'districtGeoAddress',
}
df_district.rename(columns=col_name, inplace=True)

In [5]:
col_name = {
    'address': 'wardAddress',
    'latitude': 'wardLat',
    'longitude': 'wardLon',
    'bounds': 'wardBounds',
    'geo_address': 'wardGeoAddress',
}
df_ward.rename(columns=col_name, inplace=True)

In [6]:
df = pd.read_csv(BASE_DIR / 'data/interim/convert_legacy_2025_simple.csv')

In [7]:
df = pd.merge(df, df_province, on='province', how='left')

df['districtAddress'] = df['district'] + ', ' + df['province']
df = pd.merge(df, df_district, on='districtAddress', how='left')

df['wardAddress'] = df['ward'].fillna('') + ', ' + df['district'] + ', ' + df['province']
df = pd.merge(df, df_ward, on='wardAddress', how='left')

# Choosing default new ward for divided wards

In [8]:
# Assign default new ward for divided wards
df_divided = df[df['isDividedWard'] == True].copy()
df_divided.sort_values(by=['province', 'district', 'ward', 'newWardAreaKm2'], inplace=True)
df_divided_ward = df_divided[['province', 'district', 'ward', 'wardLat', 'wardLon']].drop_duplicates()


for index, row in df_divided_ward.iterrows():
    province = row['province']
    district = row['district']
    ward = row['ward']
    ward_point = (row['wardLat'], row['wardLon'])


    new_wards = df[(df['province'] == province) & (df['district'] == district) & (df['ward'] == ward)]

    containing_points = []
    new_ward_points = []
    for new_ward_index, new_ward_row in new_wards.iterrows():
        new_ward_point = (new_ward_row['newWardLat'], new_ward_row['newWardLon'])
        new_ward_area_km2 = new_ward_row['newWardAreaKm2']
        new_ward_points.append(new_ward_point)

        is_contain = check_point_in_polygon(point=ward_point, polygon_center=new_ward_point,
                                            polygon_area_km2=new_ward_area_km2)
        if is_contain:
            containing_points.append(new_ward_point)
        df.loc[new_ward_index, 'isNewWardPolygonContainsWard'] = is_contain

    nearest_point = find_nearest_point(a_point=ward_point, list_of_b_points=new_ward_points)

    if len(containing_points) == 1:
        default_ward_point = containing_points[0]
    else:
        default_ward_point = nearest_point

    df.loc[(df['province'] == province) & (df['district'] == district) & (df['ward'] == ward) & (
                df['newWardLat'] == nearest_point[0]) & (
                       df['newWardLon'] == nearest_point[1]), 'isNearestNewWard'] = True
    df.loc[(df['province'] == province) & (df['district'] == district) & (df['ward'] == ward) & (
                df['newWardLat'] == default_ward_point[0]) & (
                       df['newWardLon'] == default_ward_point[1]), 'isDefaultNewWard'] = True

df.loc[(df['isDividedWard'] == True) & (df['isNearestNewWard'].isna()), 'isNearestNewWard'] = False
df.loc[(df['isDividedWard'] == True) & (df['isDefaultNewWard'].isna()), 'isDefaultNewWard'] = False

# Enriching data

- short name
- ~~english name~~

In [9]:
# 3321
def create_sort_34(text, level=1):
    if isinstance(text, str):
        if level == 1:
            text = re.sub(r'^Tỉnh\s|Thành phố\s|Thủ đô\s', '', text, flags=re.IGNORECASE)
        else:
            text = re.sub(r'^Phường\s|Đặc khu\s|Xã\s', '', text, flags=re.IGNORECASE)

        return text.strip()
    return text


df['newWardType'] = df['newWardType'].str.capitalize()
df.rename(columns={'newWard': 'newWardShort'}, inplace=True)
df['newWard'] = df['newWardType'] + ' ' + df['newWardShort']
df['newProvinceShort'] = df['newProvince'].apply(create_sort_34)

In [10]:
# 63
def create_sort_63(text, level=1):
    if isinstance(text, str):
        if level == 1:
            text = re.sub(r'^Tỉnh\s|Thành phố\s', '', text, flags=re.IGNORECASE)
        elif level == 2:
            if re.search(r'^Quận\s\d{1,2}', text, flags=re.IGNORECASE):
                pass
            else:
                text = re.sub(r'^Quận\s|Huyện\s|Thị xã\s|Thành phố\s', '', text, flags=re.IGNORECASE)
        else:
            if re.search(r'^Phường\s\d{1,2}', text, flags=re.IGNORECASE):
                pass
            else:
                text = re.sub(r'^Phường\s|Thị trấn\s|Xã\s', '', text, flags=re.IGNORECASE)

        return text.strip()
    return text


unit_cols = ['province', 'district', 'ward']
level_map = {
    'province': 1,
    'district': 2,
    'ward': 3
}

for col in unit_cols:
    # Create short version
    level = level_map[col]
    df[f"{col}Short"] = df[col].apply(create_sort_63, args=(level,))

In [11]:
df

,provinceCode,isMergedProvince,districtCode,districtType,wardCode,wardType,isMergedWard,isDividedWard,province,district,...,wardBounds,wardGeoAddress,isNewWardPolygonContainsWard,isNearestNewWard,isDefaultNewWard,newWard,newProvinceShort,provinceShort,districtShort,wardShort
0,1,False,1,Quận,1.0,Phường,True,False,Thành phố Hà Nội,Quận Ba Đình,...,"21.040199,105.842293 – 21.053464,105.855959","Phuc Xa, Ba Đình, Hanoi, Vietnam",NaN,NaN,NaN,Phường Hồng Hà,Hà Nội,Hà Nội,Ba Đình,Phúc Xá
1,1,False,1,Quận,4.0,Phường,True,False,Thành phố Hà Nội,Quận Ba Đình,...,"21.039976,105.836277 – 21.050836,105.850246","Truc Bach, Ba Đình, Hanoi, Vietnam",NaN,NaN,NaN,Phường Ba Đình,Hà Nội,Hà Nội,Ba Đình,Trúc Bạch
2,1,False,1,Quận,6.0,Phường,True,False,Thành phố Hà Nội,Quận Ba Đình,...,"21.035736,105.805872 – 21.047587,105.815001","Vĩnh Phúc, Ba Đình, Hà Nội, Vietnam",NaN,NaN,NaN,Phường Ngọc Hà,Hà Nội,Hà Nội,Ba Đình,Vĩnh Phúc
3,1,False,1,Quận,7.0,Phường,True,True,Thành phố Hà Nội,Quận Ba Đình,...,"21.031335,105.804777 – 21.037841,105.814554","Cống Vị, Ba Đình, Hà Nội, Vietnam",False,False,False,Phường Giảng Võ,Hà Nội,Hà Nội,Ba Đình,Cống Vị
4,1,False,1,Quận,7.0,Phường,True,True,Thành phố Hà Nội,Quận Ba Đình,...,"21.031335,105.804777 – 21.037841,105.814554","Cống Vị, Ba Đình, Hà Nội, Vietnam",True,True,True,Phường Ngọc Hà,Hà Nội,Hà Nội,Ba Đình,Cống Vị
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10597,96,True,973,Huyện,32242.0,Xã,True,True,Tỉnh Cà Mau,Huyện Ngọc Hiển,...,"8.567442,104.772921 – 8.737998,104.915485","Viên An, Ngọc Hiển District, Ca Mau, Vietnam",True,True,True,Xã Đất Mũi,Cà Mau,Cà Mau,Ngọc Hiển,Viên An
10598,96,True,973,Huyện,32244.0,Thị trấn,True,False,Tỉnh Cà Mau,Huyện Ngọc Hiển,...,"8.596467,104.967112 – 8.679712,105.061312","Rạch Gốc, Ngọc Hiển District, Ca Mau, Vietnam",NaN,NaN,NaN,Xã Phan Ngọc Hiển,Cà Mau,Cà Mau,Ngọc Hiển,Rạch Gốc
10599,96,True,973,Huyện,32245.0,Xã,True,True,Tỉnh Cà Mau,Huyện Ngọc Hiển,...,"8.598292,105.007668 – 8.701729,105.142808","Tân Ân, Ngọc Hiển, Cà Mau, Vietnam",False,False,False,Xã Đất Mũi,Cà Mau,Cà Mau,Ngọc Hiển,Tân Ân
10600,96,True,973,Huyện,32245.0,Xã,True,True,Tỉnh Cà Mau,Huyện Ngọc Hiển,...,"8.598292,105.007668 – 8.701729,105.142808","Tân Ân, Ngọc Hiển, Cà Mau, Vietnam",False,True,True,Xã Phan Ngọc Hiển,Cà Mau,Cà Mau,Ngọc Hiển,Tân Ân


## 63 provinces

In [12]:
col_63 = [
    'province',
    'district',
    'ward',

    'provinceShort',
    'districtShort',
    'wardShort',

    'districtType',
    'wardType',

    'provinceCode',
    'provinceLat',
    'provinceLon',
    'provinceBounds',
    'provinceGeoAddress',

    'districtCode',

    'districtLat',
    'districtLon',
    'districtBounds',
    'districtGeoAddress',

    'wardCode',

    'wardLat',
    'wardLon',
    'wardBounds',
    'wardGeoAddress',
]
df_63 = df[col_63].drop_duplicates().reset_index(drop=True)


In [13]:
# Add info to duplicated districtShort in a province
df_63_district_short = df_63[['province', 'district', 'districtShort']].drop_duplicates()
count_63_district_short = df_63_district_short.groupby(['province', 'districtShort']).size().reset_index(name='count').sort_values(by=['count'], ascending=False)
duplicated_63_district_short = count_63_district_short[count_63_district_short['count'] >= 2].copy()
duplicated_63_district_short['districtShortDuplicated'] = True
duplicated_63_district_short.drop(columns=['count'], inplace=True)

df_63 = pd.merge(df_63, duplicated_63_district_short, on=['province', 'districtShort'], how='left')
df_63['districtShortDuplicated'].fillna(False, inplace=True)
df_63['districtShort'] = np.where(df_63['districtShortDuplicated']==True, df_63['districtShort'] + ' (' + df_63['districtType'] + ')', df_63['districtShort'] )
df_63[df_63['districtShortDuplicated']==True]

,province,district,ward,provinceShort,districtShort,wardShort,districtType,wardType,provinceCode,provinceLat,...,districtLat,districtLon,districtBounds,districtGeoAddress,wardCode,wardLat,wardLon,wardBounds,wardGeoAddress,districtShortDuplicated
5499,Tỉnh Hà Tĩnh,Huyện Kỳ Anh,Xã Kỳ Xuân,Hà Tĩnh,Kỳ Anh (Huyện),Kỳ Xuân,Huyện,Xã,42,18.245968,...,18.064047,106.205148,"17.90839,106.014361 – 18.311613,106.338718","Kỳ Anh District, Ha Tinh, Vietnam",18757.0,18.232379,106.187502,"18.206315,106.151429 – 18.267693,106.226654","Kỳ Xuân, Kỳ Anh District, Ha Tinh, Vietnam",True
5500,Tỉnh Hà Tĩnh,Huyện Kỳ Anh,Xã Kỳ Bắc,Hà Tĩnh,Kỳ Anh (Huyện),Kỳ Bắc,Huyện,Xã,42,18.245968,...,18.064047,106.205148,"17.90839,106.014361 – 18.311613,106.338718","Kỳ Anh District, Ha Tinh, Vietnam",18760.0,18.223910,106.152213,"18.194714,106.124572 – 18.257564,106.184113","Kỳ Bắc, Kỳ Anh District, Ha Tinh, Vietnam",True
5501,Tỉnh Hà Tĩnh,Huyện Kỳ Anh,Xã Kỳ Phú,Hà Tĩnh,Kỳ Anh (Huyện),Kỳ Phú,Huyện,Xã,42,18.245968,...,18.064047,106.205148,"17.90839,106.014361 – 18.311613,106.338718","Kỳ Anh District, Ha Tinh, Vietnam",18763.0,18.204285,106.246331,"18.171752,106.220233 – 18.223286,106.280559","Kỳ Phú, Kỳ Anh District, Ha Tinh, Vietnam",True
5502,Tỉnh Hà Tĩnh,Huyện Kỳ Anh,Xã Kỳ Phong,Hà Tĩnh,Kỳ Anh (Huyện),Kỳ Phong,Huyện,Xã,42,18.245968,...,18.064047,106.205148,"17.90839,106.014361 – 18.311613,106.338718","Kỳ Anh District, Ha Tinh, Vietnam",18766.0,18.177876,106.152213,"18.130806,106.127012 – 18.217581,106.182509","Kỳ Phong, Kỳ Anh District, Ha Tinh, Vietnam",True
5503,Tỉnh Hà Tĩnh,Huyện Kỳ Anh,Xã Kỳ Tiến,Hà Tĩnh,Kỳ Anh (Huyện),Kỳ Tiến,Huyện,Xã,42,18.245968,...,18.064047,106.205148,"17.90839,106.014361 – 18.311613,106.338718","Kỳ Anh District, Ha Tinh, Vietnam",18769.0,18.186350,106.187502,"18.15116,106.169389 – 18.219765,106.213669","Kỳ Tiến, Kỳ Anh District, Ha Tinh, Vietnam",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9763,Tỉnh Hậu Giang,Thị xã Long Mỹ,Phường Vĩnh Tường,Hậu Giang,Long Mỹ (Thị xã),Vĩnh Tường,Thị xã,Phường,93,9.757898,...,9.669597,105.565024,"9.6511,105.54677 – 9.695774,105.592861","Long Mỹ, Long Mỹ District, Hau Giang, Vietnam",31475.0,9.732223,105.573697,"9.730874,105.572348 – 9.733572,105.575045","Vĩnh Tường, Long Mỹ, Hậu Giang, Vietnam",True
9764,Tỉnh Hậu Giang,Thị xã Long Mỹ,Xã Long Trị,Hậu Giang,Long Mỹ (Thị xã),Long Trị,Thị xã,Xã,93,9.757898,...,9.669597,105.565024,"9.6511,105.54677 – 9.695774,105.592861","Long Mỹ, Long Mỹ District, Hau Giang, Vietnam",31477.0,9.669597,105.565024,"9.6511,105.54677 – 9.695774,105.592861","Long Mỹ, Long Mỹ District, Hau Giang, Vietnam",True
9765,Tỉnh Hậu Giang,Thị xã Long Mỹ,Xã Long Trị A,Hậu Giang,Long Mỹ (Thị xã),Long Trị A,Thị xã,Xã,93,9.757898,...,9.669597,105.565024,"9.6511,105.54677 – 9.695774,105.592861","Long Mỹ, Long Mỹ District, Hau Giang, Vietnam",31478.0,9.669597,105.565024,"9.6511,105.54677 – 9.695774,105.592861","Long Mỹ, Long Mỹ District, Hau Giang, Vietnam",True
9766,Tỉnh Hậu Giang,Thị xã Long Mỹ,Xã Long Phú,Hậu Giang,Long Mỹ (Thị xã),Long Phú,Thị xã,Xã,93,9.757898,...,9.669597,105.565024,"9.6511,105.54677 – 9.695774,105.592861","Long Mỹ, Long Mỹ District, Hau Giang, Vietnam",31480.0,9.669597,105.565024,"9.6511,105.54677 – 9.695774,105.592861","Long Mỹ, Long Mỹ District, Hau Giang, Vietnam",True


In [14]:
count_63_ward_short = df_63.groupby(['province', 'district', 'wardShort']).size().reset_index(name='count').sort_values(by=['count'], ascending=False)
duplicated_63_ward_short = count_63_ward_short[count_63_ward_short['count'] >= 2].copy()
duplicated_63_ward_short['wardShortDuplicated'] = True
duplicated_63_ward_short.drop(columns=['count'], inplace=True)

df_63 = pd.merge(df_63, duplicated_63_ward_short, on=['province', 'district', 'wardShort'], how='left')
df_63['wardShortDuplicated'].fillna(False, inplace=True)
df_63['wardShort'] = np.where(df_63['wardShortDuplicated']==True, df_63['wardShort'] + ' (' + df_63['wardType'] + ')', df_63['wardShort'])
df_63[df_63['wardShortDuplicated']==True]

,province,district,ward,provinceShort,districtShort,wardShort,districtType,wardType,provinceCode,provinceLat,...,districtLon,districtBounds,districtGeoAddress,wardCode,wardLat,wardLon,wardBounds,wardGeoAddress,districtShortDuplicated,wardShortDuplicated
165,Thành phố Hà Nội,Huyện Gia Lâm,Thị trấn Yên Viên,Hà Nội,Gia Lâm,Yên Viên (Thị trấn),Huyện,Thị trấn,1,21.027764,...,105.958237,"20.926328,105.880257 – 21.121305,106.020073","Gia Lâm, Hanoi, Vietnam",526.0,21.084465,105.916382,"21.077122,105.902887 – 21.093539,105.928094","tt. Yên Viên, Gia Lâm, Hà Nội, Vietnam",False,True
167,Thành phố Hà Nội,Huyện Gia Lâm,Xã Yên Viên,Hà Nội,Gia Lâm,Yên Viên (Xã),Huyện,Xã,1,21.027764,...,105.958237,"20.926328,105.880257 – 21.121305,106.020073","Gia Lâm, Hanoi, Vietnam",532.0,21.079044,105.920053,"21.071766,105.904427 – 21.099095,105.930347","Yên Viên, Gia Lâm, Hanoi, Vietnam",False,True
1425,Tỉnh Lai Châu,Huyện Mường Tè,Thị trấn Mường Tè,Lai Châu,Mường Tè,Mường Tè (Thị trấn),Huyện,Thị trấn,12,22.355483,...,102.713512,"22.166967,102.320669 – 22.780852,102.986228","Mường Tè District, Lai Chau, Vietnam",3433.0,22.375023,102.811225,"22.359363,102.784524 – 22.401745,102.830057","Mường Tè, Lai Châu, Vietnam",False,True
1430,Tỉnh Lai Châu,Huyện Mường Tè,Xã Mường Tè,Lai Châu,Mường Tè,Mường Tè (Xã),Huyện,Xã,12,22.355483,...,102.713512,"22.166967,102.320669 – 22.780852,102.986228","Mường Tè District, Lai Chau, Vietnam",3445.0,22.526593,102.598660,"22.436338,102.513599 – 22.638887,102.696762","xã Mường Tè, Mường Tè, Lai Châu, Vietnam",False,True
1648,Tỉnh Sơn La,Huyện Mai Sơn,Thị trấn Hát Lót,Sơn La,Mai Sơn,Hát Lót (Thị trấn),Huyện,Thị trấn,14,21.172464,...,104.053676,"20.958818,103.697428 – 21.378427,104.292848","Mai Sơn District, Son La, Vietnam",4105.0,21.205936,104.094325,"21.166884,104.070826 – 21.219501,104.133525","Hát Lót, Mai Sơn District, Son La, Vietnam",False,True
1658,Tỉnh Sơn La,Huyện Mai Sơn,Xã Hát Lót,Sơn La,Mai Sơn,Hát Lót (Xã),Huyện,Xã,14,21.172464,...,104.053676,"20.958818,103.697428 – 21.378427,104.292848","Mai Sơn District, Son La, Vietnam",4135.0,21.179238,104.065289,"21.131582,104.018125 – 21.225742,104.120436","Hát Lót, Tp. Sơn La, Sơn La, Vietnam",False,True
1820,Tỉnh Yên Bái,Huyện Trạm Tấu,Thị trấn Trạm Tấu,Yên Bái,Trạm Tấu,Trạm Tấu (Thị trấn),Huyện,Thị trấn,15,21.683525,...,104.425753,"21.351131,104.276719 – 21.669032,104.662393","Tram Tau, Yên Bái, Vietnam",4585.0,21.514850,104.431575,"21.485418,104.38385 – 21.551932,104.4841","tt. Trạm Tấu, Trạm Tấu, Yên Bái, Vietnam",False,True
1825,Tỉnh Yên Bái,Huyện Trạm Tấu,Xã Trạm Tấu,Yên Bái,Trạm Tấu,Trạm Tấu (Xã),Huyện,Xã,15,21.683525,...,104.425753,"21.351131,104.276719 – 21.669032,104.662393","Tram Tau, Yên Bái, Vietnam",4600.0,21.514850,104.431575,"21.483747,104.385526 – 21.551368,104.485792","Trạm Tấu, Tram Tau, Yên Bái, Vietnam",False,True
2345,Tỉnh Lạng Sơn,Huyện Chi Lăng,Thị trấn Chi Lăng,Lạng Sơn,Chi Lăng,Chi Lăng (Thị trấn),Huyện,Thị trấn,20,21.856371,...,106.605553,"21.535728,106.420023 – 21.797534,106.831907","Chi Lăng District, Lạng Sơn, Vietnam",6466.0,21.574166,106.505380,"21.536284,106.467133 – 21.595512,106.531162","Chi Lăng, Chi Lăng District, Lạng Sơn, Vietnam",False,True
2363,Tỉnh Lạng Sơn,Huyện Chi Lăng,Xã Chi Lăng,Lạng Sơn,Chi Lăng,Chi Lăng (Xã),Huyện,Xã,20,21.856371,...,106.605553,"21.535728,106.420023 – 21.797534,106.831907","Chi Lăng District, Lạng Sơn, Vietnam",6523.0,21.601286,106.523054,"21.564464,106.500177 – 21.625436,106.561804","x. Chi Lăng, Chi Lăng, Lạng Sơn, Vietnam",False,True


In [15]:
df_63

,province,district,ward,provinceShort,districtShort,wardShort,districtType,wardType,provinceCode,provinceLat,...,districtLon,districtBounds,districtGeoAddress,wardCode,wardLat,wardLon,wardBounds,wardGeoAddress,districtShortDuplicated,wardShortDuplicated
0,Thành phố Hà Nội,Quận Ba Đình,Phường Phúc Xá,Hà Nội,Ba Đình,Phúc Xá,Quận,Phường,1,21.027764,...,105.826094,"21.016004,105.801279 – 21.053565,105.857015","Ba Đình, Hanoi, Vietnam",1.0,21.046798,105.848112,"21.040199,105.842293 – 21.053464,105.855959","Phuc Xa, Ba Đình, Hanoi, Vietnam",False,False
1,Thành phố Hà Nội,Quận Ba Đình,Phường Trúc Bạch,Hà Nội,Ba Đình,Trúc Bạch,Quận,Phường,1,21.027764,...,105.826094,"21.016004,105.801279 – 21.053565,105.857015","Ba Đình, Hanoi, Vietnam",4.0,21.044395,105.843174,"21.039976,105.836277 – 21.050836,105.850246","Truc Bach, Ba Đình, Hanoi, Vietnam",False,False
2,Thành phố Hà Nội,Quận Ba Đình,Phường Vĩnh Phúc,Hà Nội,Ba Đình,Vĩnh Phúc,Quận,Phường,1,21.027764,...,105.826094,"21.016004,105.801279 – 21.053565,105.857015","Ba Đình, Hanoi, Vietnam",6.0,21.041731,105.809950,"21.035736,105.805872 – 21.047587,105.815001","Vĩnh Phúc, Ba Đình, Hà Nội, Vietnam",False,False
3,Thành phố Hà Nội,Quận Ba Đình,Phường Cống Vị,Hà Nội,Ba Đình,Cống Vị,Quận,Phường,1,21.027764,...,105.826094,"21.016004,105.801279 – 21.053565,105.857015","Ba Đình, Hanoi, Vietnam",7.0,21.035280,105.809216,"21.031335,105.804777 – 21.037841,105.814554","Cống Vị, Ba Đình, Hà Nội, Vietnam",False,False
4,Thành phố Hà Nội,Quận Ba Đình,Phường Liễu Giai,Hà Nội,Ba Đình,Liễu Giai,Quận,Phường,1,21.027764,...,105.826094,"21.016004,105.801279 – 21.053565,105.857015","Ba Đình, Hanoi, Vietnam",8.0,21.037997,105.818756,"21.033027,105.812652 – 21.042991,105.821777","Liễu Giai, Ba Đình, Hà Nội, Vietnam",False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10035,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Viên An Đông,Cà Mau,Ngọc Hiển,Viên An Đông,Huyện,Xã,96,8.962410,...,104.962250,"8.408631,104.70541 – 8.796906,105.218539","Ngọc Hiển District, Ca Mau, Vietnam",32239.0,8.637734,104.950567,"8.570243,104.878836 – 8.749535,104.996595","Viên An, Ngọc Hiển District, Ca Mau, Vietnam",False,False
10036,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Viên An,Cà Mau,Ngọc Hiển,Viên An,Huyện,Xã,96,8.962410,...,104.962250,"8.408631,104.70541 – 8.796906,105.218539","Ngọc Hiển District, Ca Mau, Vietnam",32242.0,8.663728,104.833788,"8.567442,104.772921 – 8.737998,104.915485","Viên An, Ngọc Hiển District, Ca Mau, Vietnam",False,False
10037,Tỉnh Cà Mau,Huyện Ngọc Hiển,Thị trấn Rạch Gốc,Cà Mau,Ngọc Hiển,Rạch Gốc,Huyện,Thị trấn,96,8.962410,...,104.962250,"8.408631,104.70541 – 8.796906,105.218539","Ngọc Hiển District, Ca Mau, Vietnam",32244.0,8.656311,105.020676,"8.596467,104.967112 – 8.679712,105.061312","Rạch Gốc, Ngọc Hiển District, Ca Mau, Vietnam",False,False
10038,Tỉnh Cà Mau,Huyện Ngọc Hiển,Xã Tân Ân,Cà Mau,Ngọc Hiển,Tân Ân,Huyện,Xã,96,8.962410,...,104.962250,"8.408631,104.70541 – 8.796906,105.218539","Ngọc Hiển District, Ca Mau, Vietnam",32245.0,8.653504,105.090815,"8.598292,105.007668 – 8.701729,105.142808","Tân Ân, Ngọc Hiển, Cà Mau, Vietnam",False,False


In [16]:
df_63.to_csv(BASE_DIR / 'data/processed/legacy_63-province-10040-ward_with_location.csv', index=False)

## 34 provinces

In [17]:
col_34 = {
    'newProvince': 'province',
    'newWard': 'ward',
    'newProvinceShort': 'provinceShort',
    'newWardShort': 'wardShort',
    'newWardType': 'wardType',

    'newProvinceCode': 'provinceCode',
    'newProvinceLat': 'provinceLat',
    'newProvinceLon': 'provinceLon',
    'newWardCode': 'wardCode',
    'newWardLat': 'wardLat',
    'newWardLon': 'wardLon',
    'newWardAreaKm2': 'wardAreaKm2',
}
df_34 = df[col_34.keys()].drop_duplicates().reset_index(drop=True).rename(columns=col_34)

In [18]:
df_34.groupby(['province', 'wardShort']).size().reset_index(name='count').sort_values(by=['count'], ascending=False)
# No duplicated with accented name

,province,wardShort,count
0,Thành phố Cần Thơ,An Bình,1
2218,Tỉnh Quảng Ninh,Đông Ngũ,1
2208,Tỉnh Quảng Ninh,Tiên Yên,1
2209,Tỉnh Quảng Ninh,Tuần Châu,1
2210,Tỉnh Quảng Ninh,Uông Bí,1
...,...,...,...
1110,Tỉnh Hà Tĩnh,Gia Hanh,1
1111,Tỉnh Hà Tĩnh,Hoành Sơn,1
1112,Tỉnh Hà Tĩnh,Hà Huy Tập,1
1113,Tỉnh Hà Tĩnh,Hà Linh,1


In [19]:
df_34

,province,ward,provinceShort,wardShort,wardType,provinceCode,provinceLat,provinceLon,wardCode,wardLat,wardLon,wardAreaKm2
0,Thủ đô Hà Nội,Phường Hồng Hà,Hà Nội,Hồng Hà,Phường,1,21.0001,105.698,42.0,21.05670,105.845,15.09
1,Thủ đô Hà Nội,Phường Ba Đình,Hà Nội,Ba Đình,Phường,1,21.0001,105.698,3.0,21.03860,105.838,2.97
2,Thủ đô Hà Nội,Phường Ngọc Hà,Hà Nội,Ngọc Hà,Phường,1,21.0001,105.698,61.0,21.03810,105.816,2.68
3,Thủ đô Hà Nội,Phường Giảng Võ,Hà Nội,Giảng Võ,Phường,1,21.0001,105.698,30.0,21.02750,105.814,2.60
4,Thủ đô Hà Nội,Phường Hoàn Kiếm,Hà Nội,Hoàn Kiếm,Phường,1,21.0001,105.698,39.0,21.03200,105.850,1.91
...,...,...,...,...,...,...,...,...,...,...,...,...
3316,Tỉnh Cà Mau,Xã Phú Tân,Cà Mau,Phú Tân,Xã,34,9.1362,105.182,3331.0,8.92852,104.846,101.70
3317,Tỉnh Cà Mau,Xã Nguyễn Việt Khái,Cà Mau,Nguyễn Việt Khái,Xã,34,9.1362,105.182,3324.0,8.86914,104.930,129.90
3318,Tỉnh Cà Mau,Xã Tân Ân,Cà Mau,Tân Ân,Xã,34,9.1362,105.182,3337.0,8.72769,105.078,218.30
3319,Tỉnh Cà Mau,Xã Phan Ngọc Hiển,Cà Mau,Phan Ngọc Hiển,Xã,34,9.1362,105.182,3327.0,8.64616,104.943,237.70


In [20]:
df_34.to_csv(BASE_DIR / 'data/processed/2025_34-province-3221-ward_with_location.csv', index=False)

## Convert

In [21]:
df.drop(columns=['districtShort', 'wardShort'], inplace=True)

In [22]:
df = pd.merge(df, df_63[['province', 'district', 'ward', 'districtShort', 'wardShort', 'districtShortDuplicated', 'wardShortDuplicated']], on=['province', 'district', 'ward'], how='left')

In [23]:
df

,provinceCode,isMergedProvince,districtCode,districtType,wardCode,wardType,isMergedWard,isDividedWard,province,district,...,isNewWardPolygonContainsWard,isNearestNewWard,isDefaultNewWard,newWard,newProvinceShort,provinceShort,districtShort,wardShort,districtShortDuplicated,wardShortDuplicated
0,1,False,1,Quận,1.0,Phường,True,False,Thành phố Hà Nội,Quận Ba Đình,...,NaN,NaN,NaN,Phường Hồng Hà,Hà Nội,Hà Nội,Ba Đình,Phúc Xá,False,False
1,1,False,1,Quận,4.0,Phường,True,False,Thành phố Hà Nội,Quận Ba Đình,...,NaN,NaN,NaN,Phường Ba Đình,Hà Nội,Hà Nội,Ba Đình,Trúc Bạch,False,False
2,1,False,1,Quận,6.0,Phường,True,False,Thành phố Hà Nội,Quận Ba Đình,...,NaN,NaN,NaN,Phường Ngọc Hà,Hà Nội,Hà Nội,Ba Đình,Vĩnh Phúc,False,False
3,1,False,1,Quận,7.0,Phường,True,True,Thành phố Hà Nội,Quận Ba Đình,...,False,False,False,Phường Giảng Võ,Hà Nội,Hà Nội,Ba Đình,Cống Vị,False,False
4,1,False,1,Quận,7.0,Phường,True,True,Thành phố Hà Nội,Quận Ba Đình,...,True,True,True,Phường Ngọc Hà,Hà Nội,Hà Nội,Ba Đình,Cống Vị,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10597,96,True,973,Huyện,32242.0,Xã,True,True,Tỉnh Cà Mau,Huyện Ngọc Hiển,...,True,True,True,Xã Đất Mũi,Cà Mau,Cà Mau,Ngọc Hiển,Viên An,False,False
10598,96,True,973,Huyện,32244.0,Thị trấn,True,False,Tỉnh Cà Mau,Huyện Ngọc Hiển,...,NaN,NaN,NaN,Xã Phan Ngọc Hiển,Cà Mau,Cà Mau,Ngọc Hiển,Rạch Gốc,False,False
10599,96,True,973,Huyện,32245.0,Xã,True,True,Tỉnh Cà Mau,Huyện Ngọc Hiển,...,False,False,False,Xã Đất Mũi,Cà Mau,Cà Mau,Ngọc Hiển,Tân Ân,False,False
10600,96,True,973,Huyện,32245.0,Xã,True,True,Tỉnh Cà Mau,Huyện Ngọc Hiển,...,False,True,True,Xã Phan Ngọc Hiển,Cà Mau,Cà Mau,Ngọc Hiển,Tân Ân,False,False


In [24]:
cols = [
    'provinceCode',
    'isMergedProvince',
    'districtCode',
    'districtType',
    'districtShortDuplicated',
    'wardCode',
    'wardType',
    'wardShortDuplicated',
    'isMergedWard',
    'isDividedWard',

    'province',
    'district',
    'ward',

    'provinceShort',
    'districtShort',
    'wardShort',

    'newProvince',
    'newWard',
    'newWardShort',
    'newWardType',
    'newProvinceCode',
    'newProvinceLat',
    'newProvinceLon',
    'newWardCode',
    'newWardLat',
    'newWardLon',
    'newWardAreaKm2',
    'provinceLat',
    'provinceLon',
    'provinceBounds',
    'provinceGeoAddress',
    'districtAddress',
    'districtLat',
    'districtLon',
    'districtBounds',
    'districtGeoAddress',
    'wardAddress',
    'wardLat',
    'wardLon',
    'wardBounds',
    'wardGeoAddress',
    'isNewWardPolygonContainsWard',
    'isNearestNewWard',
    'isDefaultNewWard',
    'newProvinceShort',

]
df = df[cols]

In [25]:
df

,provinceCode,isMergedProvince,districtCode,districtType,districtShortDuplicated,wardCode,wardType,wardShortDuplicated,isMergedWard,isDividedWard,...,districtGeoAddress,wardAddress,wardLat,wardLon,wardBounds,wardGeoAddress,isNewWardPolygonContainsWard,isNearestNewWard,isDefaultNewWard,newProvinceShort
0,1,False,1,Quận,False,1.0,Phường,False,True,False,...,"Ba Đình, Hanoi, Vietnam","Phường Phúc Xá, Quận Ba Đình, Thành phố Hà Nội",21.046798,105.848112,"21.040199,105.842293 – 21.053464,105.855959","Phuc Xa, Ba Đình, Hanoi, Vietnam",NaN,NaN,NaN,Hà Nội
1,1,False,1,Quận,False,4.0,Phường,False,True,False,...,"Ba Đình, Hanoi, Vietnam","Phường Trúc Bạch, Quận Ba Đình, Thành phố Hà Nội",21.044395,105.843174,"21.039976,105.836277 – 21.050836,105.850246","Truc Bach, Ba Đình, Hanoi, Vietnam",NaN,NaN,NaN,Hà Nội
2,1,False,1,Quận,False,6.0,Phường,False,True,False,...,"Ba Đình, Hanoi, Vietnam","Phường Vĩnh Phúc, Quận Ba Đình, Thành phố Hà Nội",21.041731,105.809950,"21.035736,105.805872 – 21.047587,105.815001","Vĩnh Phúc, Ba Đình, Hà Nội, Vietnam",NaN,NaN,NaN,Hà Nội
3,1,False,1,Quận,False,7.0,Phường,False,True,True,...,"Ba Đình, Hanoi, Vietnam","Phường Cống Vị, Quận Ba Đình, Thành phố Hà Nội",21.035280,105.809216,"21.031335,105.804777 – 21.037841,105.814554","Cống Vị, Ba Đình, Hà Nội, Vietnam",False,False,False,Hà Nội
4,1,False,1,Quận,False,7.0,Phường,False,True,True,...,"Ba Đình, Hanoi, Vietnam","Phường Cống Vị, Quận Ba Đình, Thành phố Hà Nội",21.035280,105.809216,"21.031335,105.804777 – 21.037841,105.814554","Cống Vị, Ba Đình, Hà Nội, Vietnam",True,True,True,Hà Nội
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10597,96,True,973,Huyện,False,32242.0,Xã,False,True,True,...,"Ngọc Hiển District, Ca Mau, Vietnam","Xã Viên An, Huyện Ngọc Hiển, Tỉnh Cà Mau",8.663728,104.833788,"8.567442,104.772921 – 8.737998,104.915485","Viên An, Ngọc Hiển District, Ca Mau, Vietnam",True,True,True,Cà Mau
10598,96,True,973,Huyện,False,32244.0,Thị trấn,False,True,False,...,"Ngọc Hiển District, Ca Mau, Vietnam","Thị trấn Rạch Gốc, Huyện Ngọc Hiển, Tỉnh Cà Mau",8.656311,105.020676,"8.596467,104.967112 – 8.679712,105.061312","Rạch Gốc, Ngọc Hiển District, Ca Mau, Vietnam",NaN,NaN,NaN,Cà Mau
10599,96,True,973,Huyện,False,32245.0,Xã,False,True,True,...,"Ngọc Hiển District, Ca Mau, Vietnam","Xã Tân Ân, Huyện Ngọc Hiển, Tỉnh Cà Mau",8.653504,105.090815,"8.598292,105.007668 – 8.701729,105.142808","Tân Ân, Ngọc Hiển, Cà Mau, Vietnam",False,False,False,Cà Mau
10600,96,True,973,Huyện,False,32245.0,Xã,False,True,True,...,"Ngọc Hiển District, Ca Mau, Vietnam","Xã Tân Ân, Huyện Ngọc Hiển, Tỉnh Cà Mau",8.653504,105.090815,"8.598292,105.007668 – 8.701729,105.142808","Tân Ân, Ngọc Hiển, Cà Mau, Vietnam",False,True,True,Cà Mau


In [26]:
df.to_csv(BASE_DIR / 'data/processed/convert_legacy_2025_with_location_and_default_ward.csv', index=False)